<a href="https://colab.research.google.com/github/mjgpinheiro/Physics_models/blob/main/Modular_Operators_Construction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# --- Modular operators + NLS (2D torus, unit-square geometry for Step 1) ---
import numpy as np

# ---- Arithmetic helpers ----
def _sigma(n, k=1):
    s=0
    r=int(n**0.5)
    for d in range(1, r+1):
        if n % d == 0:
            s += d**k
            q = n//d
            if q != d: s += q**k
    return s

def E2(tau, N=200):
    q = np.exp(2j*np.pi*tau); s = 1.0+0j; qn=q
    for n in range(1, N+1):
        s += -24.0*_sigma(n,1)*qn
        qn *= q
    return s

def E4(tau, N=200):
    q = np.exp(2j*np.pi*tau); s = 1.0+0j; qn=q
    for n in range(1, N+1):
        s += 240.0*_sigma(n,3)*qn
        qn *= q
    return s

def E6(tau, N=200):
    q = np.exp(2j*np.pi*tau); s = 1.0+0j; qn=q
    for n in range(1, N+1):
        s += -504.0*_sigma(n,5)*qn
        qn *= q
    return s

# Ramanujan identities for derivatives (exact, fast, stable)
def dE2_dtau(tau): return 2j*np.pi*(E2(tau)**2 - E4(tau))/12.0
def dE4_dtau(tau): return 2j*np.pi*(E2(tau)*E4(tau) - E6(tau))/3.0
def dE6_dtau(tau): return 2j*np.pi*(E2(tau)*E6(tau) - E4(tau)**2)/2.0

def rankin_cohen_1(f, df, g, dg, k, l):
    # [f,g]_1 = (1/(2π i)) (k f g' - l f' g)
    return (k*f*dg - l*df*g) / (2j*np.pi)

# ---- Grid, spectrum, and Hecke projector ----
def spectral_operators(Ny, Nx):
    ky = np.fft.fftfreq(Ny)*Ny
    kx = np.fft.fftfreq(Nx)*Nx
    KX, KY = np.meshgrid(kx, ky)
    k2 = (2*np.pi)**2 * (KX**2 + KY**2)
    return KX, KY, k2

def hecke_mask(Ny, Nx, n):
    my = np.fft.fftfreq(Ny)*Ny
    mx = np.fft.fftfreq(Nx)*Nx
    MX, MY = np.meshgrid(mx, my)
    M = (np.mod(MX, n)==0) & (np.mod(MY, n)==0)
    return M.astype(float)

# ---- Modular NLS stepper (Strang split) ----
def modular_nls(psi0, steps=200, dt=1e-3, g=1.0,
                tau=0.5+0.8660254j, k_weight=2, eps=0.2, zeta=0.1, eta=0.5, n_hecke=2):
    psi = psi0.astype(complex).copy()
    Ny, Nx = psi.shape
    KX, KY, k2  = spectral_operators(Ny, Nx)
    M           = hecke_mask(Ny, Nx, n_hecke)
    # D_k on a field (no explicit tau-dependence): a scalar multiplier
    beta = -(k_weight/12.0) * E2(tau)
    # RC drift coefficient from (E4, E6)
    e4,e6 = E4(tau), E6(tau)
    de4,de6 = dE4_dtau(tau), dE6_dtau(tau)
    Ctau = rankin_cohen_1(e4, de4, e6, de6, 4, 6)   # complex scalar
    # Linear eigenvalue in Fourier domain
    eig = k2 + zeta*Ctau*(1j*2*np.pi*KX) + eps*beta + eta*(M - 1.0)
    prop = np.exp(-1j * eig * dt)
    for _ in range(steps):
        # half linear
        psi_hat = np.fft.fft2(psi); psi_hat *= prop; psi = np.fft.ifft2(psi_hat)
        # nonlinear
        psi *= np.exp(-1j * g * np.abs(psi)**2 * dt)
        # half linear
        psi_hat = np.fft.fft2(psi); psi_hat *= prop; psi = np.fft.ifft2(psi_hat)
    return psi

# ---- Run a demo ----
Ny,Nx = 128,128
y = np.linspace(0,1,Ny,endpoint=False); x = np.linspace(0,1,Nx,endpoint=False)
X,Y = np.meshgrid(x,y)
psi0 = np.exp(-120*((X-0.5)**2 + (Y-0.5)**2)) * np.exp(2j*np.pi*(8*X + 0*Y))  # bump + carrier

psiT = modular_nls(
    psi0, steps=600, dt=7.5e-4, g=+1.0,
    tau=np.exp(1j*np.pi/3),     # hexagonal torus as coefficient source
    k_weight=2, eps=0.25,       # Ramanujan–Serre (E2) strength
    zeta=0.15,                  # RC guided drift
    eta=0.8, n_hecke=3          # Hecke pruning to multiples of 3
)

# Inspect energy in Fourier (expect pruning to kx,ky ≡ 0 mod 3)
spec = np.log10(1e-16 + np.abs(np.fft.fftshift(np.fft.fft2(psiT)))**2)
print(float(spec.max()), float(spec.mean()))


304.3277831462666 302.611001298674
